In [ ]:
# Mixscape for KO effect analysis
# Load packages
library(Seurat)
library(ggplot2)
library(patchwork)
library(scales)
library(dplyr)
library(reshape2)


In [ ]:
##Read in data
seurat <- readRDS('/seurat.rds')


In [ ]:
# Calculate perturbation signature (PRTB).
seurat <- CalcPerturbSig(
  object = seurat, 
  assay = "RNA", 
  slot = "data", 
  gd.class ="KO_call", 
  nt.cell.class = "Ctrl", 
  reduction = "pca", 
  ndims = 30, 
  num.neighbors = 20, 
  new.assay.name = "PRTB")


Processing rep1



In [6]:
# Prepare PRTB assay for dimensionality reduction: 
# Normalize data, find variable features and center data.
DefaultAssay(seurat) <- 'PRTB'

# Use variable features from RNA assay.
VariableFeatures(seurat) <- VariableFeatures(seurat[["RNA"]])
seurat <- ScaleData(seurat, do.scale = F, do.center = T)

# Run PCA to reduce the dimensionality of the data.
seurat <- RunPCA(seurat, reduction.key = 'prtbpca', reduction.name = 'prtbpca')

# Run UMAP to visualize clustering in 2-D.
seurat <- RunUMAP(
  object = seurat, 
  dims = 1:30, 
  reduction = 'prtbpca', 
  reduction.key = 'prtbumap', 
  reduction.name = 'prtbumap')


Centering data matrix

Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from prtbpca to prtbpca_”
prtbpca_ 1 
Positive:  NRG1, ADARB2, CNTNAP2, CNTN5, LRP1B, PTPRD, PLCB1, MT-CYB, MT-CO1, FAM155A 
	   RIMS2, ROBO1, RBMS3, CTNND2, MT-CO2, CCSER1, GRIK2, KCNIP4, MT-ND1, MT-ND4 
	   MT-ND4L, MT-CO3, ANK2, EPHA6, ZNF804A, NRG3, CTNNA2, MT-ATP6, ADGRL3, NBEA 
Negative:  RPS19, NNAT, MDK, BEX1, TPT1, HMGN2, ID3, DDIT4, B2M, CDKN1C 
	   FTL, TMSB15A, RPS18, HMGB2, SLC22A17, HES4, H2AFZ, BBC3, ISYNA1, HLA-B 
	   BTG1, ATF3, RPRM, SNRPB, TEX14, HLA-C, TCEAL2, HMGB1, RPS27L, BAMBI 
prtbpca_ 2 
Positive:  AC109466.1, NNAT, NCAM2, AC008415.1, ATP1B1, PRKG1, GRIK2, CNTNAP2, PLXNA4, KCND2 
	   CPNE4, SST, MSX1, EYA1, B2M, ROBO2, KITLG, THBS1, CADM1, PDE4DIP 
	   FHIT, PTPRD, BCHE, ZBTB20, PDE1C, RMST, ZNF536, SRBD1, BTBD3, NRG3 
Negative:  KCNIP4, DLC1, NTS, LMO3, PCDH7, SEMA3D, CNTN5, SPOCK1, C1QL1, PPP2R2B 
	   SCG2, SLC5A7, SORBS2, SNCA, 

In [8]:
install.packages('mixtools')

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [9]:
# Run mixscape.
seurat <- RunMixscape(
  object = seurat, 
  assay = "PRTB", 
  slot = "scale.data", 
  labels = "KO_call", 
  nt.class.name = "Ctrl", 
  min.de.genes = 5, 
  iter.num = 10, 
  de.assay = "RNA", 
  verbose = F,
  prtb.type = "KO")


number of iterations= 923 
number of iterations= 498 
number of iterations= 667 
number of iterations= 735 
number of iterations= 730 
number of iterations= 757 
number of iterations= 739 
number of iterations= 715 
number of iterations= 737 
number of iterations= 830 
number of iterations= 23 
number of iterations= 14 
number of iterations= 97 
number of iterations= 94 
number of iterations= 102 
number of iterations= 103 
number of iterations= 127 
number of iterations= 78 
number of iterations= 67 
WARNING! NOT CONVERGENT! 
number of iterations= 5000 
number of iterations= 573 
number of iterations= 662 
number of iterations= 617 
number of iterations= 606 
number of iterations= 644 
number of iterations= 675 


In [ ]:
df <- seurat@meta.data %>% group_by(KO_call) %>% count(mixscape_class.global)

KO_call,mixscape_class.global,n
<chr>,<chr>,<int>
Ctrl,Ctrl,523
LHX9,KO,209
LHX9,NP,329
MSX1,KO,277
MSX1,NP,182
SMAD9,KO,426
SMAD9,NP,192
TLX2,KO,186
TLX2,NP,295


In [ ]:
# Calculate the total count for each KO_call
total_counts <- df %>%
  group_by(KO_call) %>%
  summarise(total = sum(n))

total_counts

KO_call,total
<chr>,<int>
Ctrl,523
LHX9,538
MSX1,459
SMAD9,618
TLX2,481
ZBTB20,548


In [ ]:
# Calculate the count for each KO_call in the KO group
ko_counts <- df %>%
  filter(mixscape_class.global == "KO") %>%
  select(KO_call, n_ko = n)

ko_counts


KO_call,n_ko
<chr>,<int>
LHX9,209
MSX1,277
SMAD9,426
TLX2,186
ZBTB20,308


In [ ]:
# Join the data frames and calculate the percentage
final_df <- total_counts %>%
  left_join(ko_counts, by = "KO_call") %>%
  mutate(percent_in_KO = (n_ko / total) * 100)

final_df

KO_call,total,n_ko,percent_in_KO
<chr>,<int>,<int>,<dbl>
Ctrl,523,NA,NA
LHX9,538,209,38.84758
MSX1,459,277,60.34858
SMAD9,618,426,68.93204
TLX2,481,186,38.66944
ZBTB20,548,308,56.20438


In [ ]:
library(scales) # Required for number formatting

# List of target genes to plot
target_genes <- c("LHX9", "MSX1", "TLX2", "SMAD9", "ZBTB20")

# Loop through each gene to create and save the plots
for (gene in target_genes) {
  # Define the file path for each gene's plot
  file_path <- paste0('Mixscape/', gene, '.pdf')
  
  # Open a PDF device for plotting
  pdf(file = file_path, width = 4, height = 3)
  
  # Generate the plot for the current gene
  plot <- PlotPerturbScore(
    object = seurat,
    target.gene.class = "KO_call",
    target.gene.ident = c(gene),
    mixscape.class = "mixscape_class",
    col = "coral2"
  ) + NoLegend()

  # Modify the plot to format the y-axis
  plot <- plot + scale_y_continuous(labels = scales::number_format(accuracy = 0.1))
  
  # Print the plot to the PDF
  print(plot)
  
  # Close the PDF device
  dev.off()
}
